# Signal vs. background classification with NEW full MC

In [1]:
%matplotlib inline
from matplotlib.patches         import Ellipse
import matplotlib.pyplot as plt
import numpy  as np
import random as rd
import tables as tb
import h5py
import keras.backend.tensorflow_backend as K

from __future__  import print_function
from scipy.stats import threshold

from keras.models               import Model, load_model
from keras.layers               import Input, Dense, MaxPooling3D, AveragePooling3D, Convolution3D, Activation, Dropout, merge
from keras.layers.normalization import BatchNormalization
from keras.optimizers           import SGD, Adam, Nadam         
#from keras.callbacks            import ReduceLROnPlateau !!!!!
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.layers.core          import Flatten
from keras                      import callbacks
from keras.regularizers         import l2, activity_l2

Using TensorFlow backend.


Load Data

In [2]:
## TRAINING EVENTS
f_start = 25
f_end = 50
# Read in the signal events.
print("Reading signal events...")
for fn in range(f_start,f_end):
    print("-- Reading file {0}".format(fn))
    s_dat = tb.open_file("/home/jrenner/data/classification/bb_1M_v0_08_07/hdf5_maps_NEW_training_MC_si_{0}.h5".format(fn), 'r')
    if(fn == f_start):
        s_array = np.array(s_dat.root.maps)
        s_energies = np.array(s_dat.root.energies)
    else:
        s_array = np.concatenate([s_array,np.array(s_dat.root.maps)])
        s_energies = np.concatenate([s_energies,np.array(s_dat.root.energies)])
        
# Read in the background events.
print("Reading background events...")
for fn in range(f_start,f_end):
    print("-- Reading file {0}".format(fn))
    b_dat = tb.open_file("/home/jrenner/data/classification/se_1M_v0_08_07/hdf5_maps_NEW_training_MC_bg_{0}.h5".format(fn), 'r')
    if(fn == f_start):
        b_array = np.array(b_dat.root.maps)
        b_energies = np.array(b_dat.root.energies)
    else:
        b_array = np.concatenate([b_array,np.array(b_dat.root.maps)])
        b_energies = np.concatenate([b_energies,np.array(b_dat.root.energies)])
        
print("Read {0} signal events and {1} background events.".format(len(s_array),len(b_array)))

Reading signal events...
-- Reading file 25
-- Reading file 26
-- Reading file 27
-- Reading file 28
-- Reading file 29
-- Reading file 30
-- Reading file 31
-- Reading file 32
-- Reading file 33
-- Reading file 34
-- Reading file 35
-- Reading file 36
-- Reading file 37
-- Reading file 38
-- Reading file 39
-- Reading file 40
-- Reading file 41
-- Reading file 42
-- Reading file 43
-- Reading file 44
-- Reading file 45
-- Reading file 46
-- Reading file 47
-- Reading file 48
-- Reading file 49
Reading background events...
-- Reading file 25
-- Reading file 26
-- Reading file 27
-- Reading file 28
-- Reading file 29
-- Reading file 30
-- Reading file 31
-- Reading file 32
-- Reading file 33
-- Reading file 34
-- Reading file 35
-- Reading file 36
-- Reading file 37
-- Reading file 38
-- Reading file 39
-- Reading file 40
-- Reading file 41
-- Reading file 42
-- Reading file 43
-- Reading file 44
-- Reading file 45
-- Reading file 46
-- Reading file 47
-- Reading file 48
-- Reading file

In [3]:
# Dimensions and numbers of events
Ntrain = 40000     # number of training events per sample
Ntot = 44000
xdim = 20
ydim = 20
zdim = 60

In [4]:
# Set up training (t) and validation (v) datasets.
x_t = s_array[:Ntrain]
x_v = s_array[Ntrain:Ntot]
y_t = np.ones([Ntrain, 1])
y_v = np.ones([Ntot-Ntrain, 1])

print("Concatenating datasets...")
x_t = np.concatenate([x_t, b_array[:Ntrain]])
x_v = np.concatenate([x_v, b_array[Ntrain:Ntot]])
y_bt = np.zeros([Ntrain, 1])
y_t = np.concatenate([y_t, y_bt])
y_bv = np.zeros([Ntot-Ntrain, 1])
y_v = np.concatenate([y_v, y_bv])

Concatenating datasets...


In [5]:
# Include the final dimension (single-channel).
print("Reshaping...")
#x_t = np.expand_dims(x_t, axis=1)
#x_v = np.expand_dims(x_v, axis=1)
x_t = np.reshape(x_t, (len(x_t), xdim, ydim, zdim, 1))
x_v = np.reshape(x_v, (len(x_v), xdim, ydim, zdim, 1))
print("Prepared", len(x_t), "training events, ", len(x_v), "validation events")

Reshaping...
Prepared 80000 training events,  8000 validation events


In [ ]:
# Remove noise (simple cut).
nthr = 5.0e-5
inoise = x_t < nthr
x_t[inoise] = 0

inoise = x_v < nthr
x_v[inoise] = 0

In [ ]:
# Remove noise (complex cut).
nsum_thr = 0.01

# Remove noise from training events.
print("Removing noise from training events...")
for enum in range(len(x_t)):
    for nx in range(xdim):
        for ny in range(ydim):
            sval = np.sum(x_t[enum,nx,ny,:])
            if(sval < nsum_thr):
                x_t[enum,nx,ny,:] = 0
                
# Remove noise from validation events.
print("Removing noise from validation events...")
for enum in range(len(x_v)):
    for nx in range(xdim):
        for ny in range(ydim):
            sval = np.sum(x_v[enum,nx,ny,:])
            if(sval < nsum_thr):
                x_v[enum,nx,ny,:] = 0

In [9]:
lmodel = True
model_type = 0

with K.tf.device('/gpu:1'):
    K.set_session(K.tf.Session(config=K.tf.ConfigProto(allow_soft_placement=True, log_device_placement=True)))
    
    if(lmodel):
        incep = load_model('models/conv3d_classifier_20.h5')
    elif(model_type == 0):

        inputs = Input(shape=(xdim, ydim, zdim, 1))
        cinputs = Convolution3D(256, 5, 5, 5, border_mode='same', subsample=(4, 4, 4), activation='relu',init='lecun_uniform', W_regularizer=l2(0.001))(inputs)
        cinputs = MaxPooling3D(pool_size=(3, 3, 3), strides=(2, 2, 2), border_mode='same', dim_ordering='default')(cinputs)
        cinputs = BatchNormalization(epsilon=1e-05, mode=0, axis=4, momentum=0.99, weights=None, beta_init='zero', gamma_init='one', gamma_regularizer=None, beta_regularizer=None)(cinputs)
        cinputs = Convolution3D(64, 1, 1, 1, border_mode='same', subsample=(1, 1, 1), activation='relu',init='lecun_uniform', W_regularizer=l2(0.001))(cinputs)
        cinputs = Convolution3D(128, 2, 2, 3, border_mode='same', subsample=(2, 2, 3), activation='relu',init='lecun_uniform', W_regularizer=l2(0.001))(cinputs)
        cinputs = BatchNormalization(epsilon=1e-05, mode=0, axis=4, momentum=0.99, weights=None, beta_init='zero', gamma_init='one', gamma_regularizer=None, beta_regularizer=None)(cinputs)
        cinputs = Convolution3D(128, 1, 1, 1, border_mode='same', subsample=(1, 1, 1), activation='relu',init='lecun_uniform', W_regularizer=l2(0.001))(cinputs)
        cinputs = MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), border_mode='same', dim_ordering='default')(cinputs)
        f1 = Flatten()(cinputs)
        f1 = Dense(output_dim=128, activation='relu', init='lecun_uniform', W_regularizer=l2(0.001))(f1)
        f1 = Dropout(.7)(f1)

        inc_output = Dense(output_dim=1, activation='sigmoid',init='normal', W_regularizer=l2(0.001))(f1)
        incep = Model(inputs, inc_output)

        incep.compile(loss='binary_crossentropy',
                      optimizer=Nadam(lr=0.000005, beta_1=0.9, beta_2=0.999,
                                      epsilon=1e-08, schedule_decay=0.1), metrics=['accuracy'])
    elif(model_type == 1):
        
        # Input layer
        inputs = Input(shape=(xdim, ydim, zdim, 1))
        
        # Initial convolution and normalization.
        cinputs = Convolution3D(64, 4, 4, 4, border_mode='same', subsample=(2, 2, 2), activation='relu',init='lecun_uniform', W_regularizer=l2(0.004))(inputs)
        cinputs = MaxPooling3D(pool_size=(3, 3, 3), strides=(2, 2, 2), border_mode='same', dim_ordering='default')(cinputs)
        cinputs = BatchNormalization(epsilon=1e-05, mode=0, axis=4, momentum=0.99, weights=None, beta_init='zero', gamma_init='one', gamma_regularizer=None, beta_regularizer=None)(cinputs)
        
        # 2 inception layers
        i1 = Convolution3D(64, 1, 1, 1, border_mode='same', subsample=(1, 1, 1), activation='relu',init='lecun_uniform', W_regularizer=l2(0.004))(cinputs)
        i2_1 = Convolution3D(32, 1, 1, 1, border_mode='same', subsample=(1, 1, 1), activation='relu',init='lecun_uniform', W_regularizer=l2(0.004))(cinputs)
        i2_2 = Convolution3D(128, 3, 3, 3, border_mode='same', subsample=(1, 1, 1), activation='relu',init='lecun_uniform', W_regularizer=l2(0.004))(i2_1)
        i3_1 = Convolution3D(32, 1, 1, 1, border_mode='same', subsample=(1, 1, 1), activation='relu',init='lecun_uniform', W_regularizer=l2(0.004))(cinputs)
        i3_2 = Convolution3D(64, 5, 5, 5, border_mode='same', subsample=(1, 1, 1), activation='relu',init='lecun_uniform', W_regularizer=l2(0.004))(i3_1)
        i4_1 = MaxPooling3D(pool_size=(3, 3, 3), strides=(1, 1, 1), border_mode='same', dim_ordering='default')(cinputs)
        i4_2 = Convolution3D(64, 1, 1, 1, border_mode='same', subsample=(1, 1, 1), activation='relu',init='lecun_uniform', W_regularizer=l2(0.004))(i4_1)
        cinputs = merge([i1, i2_2, i3_2, i4_2], mode='concat', concat_axis=4)
        
        i1 = Convolution3D(64, 1, 1, 1, border_mode='same', subsample=(1, 1, 1), activation='relu',init='lecun_uniform', W_regularizer=l2(0.004))(cinputs)
        i2_1 = Convolution3D(32, 1, 1, 1, border_mode='same', subsample=(1, 1, 1), activation='relu',init='lecun_uniform', W_regularizer=l2(0.004))(cinputs)
        i2_2 = Convolution3D(128, 3, 3, 3, border_mode='same', subsample=(1, 1, 1), activation='relu',init='lecun_uniform', W_regularizer=l2(0.004))(i2_1)
        i3_1 = Convolution3D(32, 1, 1, 1, border_mode='same', subsample=(1, 1, 1), activation='relu',init='lecun_uniform', W_regularizer=l2(0.004))(cinputs)
        i3_2 = Convolution3D(64, 5, 5, 5, border_mode='same', subsample=(1, 1, 1), activation='relu',init='lecun_uniform', W_regularizer=l2(0.004))(i3_1)
        i4_1 = MaxPooling3D(pool_size=(3, 3, 3), strides=(1, 1, 1), border_mode='same', dim_ordering='default')(cinputs)
        i4_2 = Convolution3D(64, 1, 1, 1, border_mode='same', subsample=(1, 1, 1), activation='relu',init='lecun_uniform', W_regularizer=l2(0.004))(i4_1)
        cinputs = merge([i1, i2_2, i3_2, i4_2], mode='concat', concat_axis=4)
        
        # Max pooling
        cinputs = MaxPooling3D(pool_size=(3, 3, 3), strides=(2, 2, 1), border_mode='same', dim_ordering='default')(cinputs)
        
        # 2 more inception layers
        #i1 = Convolution3D(32, 1, 1, 1, border_mode='same', subsample=(1, 1, 1), activation='relu',init='lecun_uniform')(cinputs)
        #i2_1 = Convolution3D(16, 1, 1, 1, border_mode='same', subsample=(1, 1, 1), activation='relu',init='lecun_uniform')(cinputs)
        #i2_2 = Convolution3D(64, 3, 3, 3, border_mode='same', subsample=(1, 1, 1), activation='relu',init='lecun_uniform')(i2_1)
        #i3_1 = Convolution3D(8, 1, 1, 1, border_mode='same', subsample=(1, 1, 1), activation='relu',init='lecun_uniform')(cinputs)
        #i3_2 = Convolution3D(32, 5, 5, 5, border_mode='same', subsample=(1, 1, 1), activation='relu',init='lecun_uniform')(i3_1)
        #i4_1 = MaxPooling3D(pool_size=(3, 3, 3), strides=(1, 1, 1), border_mode='same', dim_ordering='default')(cinputs)
        #i4_2 = Convolution3D(32, 1, 1, 1, border_mode='same', subsample=(1, 1, 1), activation='relu',init='lecun_uniform')(i4_1)
        #cinputs = merge([i1, i2_2, i3_2, i4_2], mode='concat', concat_axis=4)
        
        #i1 = Convolution3D(32, 1, 1, 1, border_mode='same', subsample=(1, 1, 1), activation='relu',init='lecun_uniform')(cinputs)
        #i2_1 = Convolution3D(16, 1, 1, 1, border_mode='same', subsample=(1, 1, 1), activation='relu',init='lecun_uniform')(cinputs)
        #i2_2 = Convolution3D(64, 3, 3, 3, border_mode='same', subsample=(1, 1, 1), activation='relu',init='lecun_uniform')(i2_1)
        #i3_1 = Convolution3D(8, 1, 1, 1, border_mode='same', subsample=(1, 1, 1), activation='relu',init='lecun_uniform')(cinputs)
        #i3_2 = Convolution3D(32, 5, 5, 5, border_mode='same', subsample=(1, 1, 1), activation='relu',init='lecun_uniform')(i3_1)
        #i4_1 = MaxPooling3D(pool_size=(3, 3, 3), strides=(1, 1, 1), border_mode='same', dim_ordering='default')(cinputs)
        #i4_2 = Convolution3D(32, 1, 1, 1, border_mode='same', subsample=(1, 1, 1), activation='relu',init='lecun_uniform')(i4_1)
        #cinputs = merge([i1, i2_2, i3_2, i4_2], mode='concat', concat_axis=4)
        
        # Max pooling
        #cinputs = MaxPooling3D(pool_size=(3, 3, 3), strides=(2, 2, 2), border_mode='same', dim_ordering='default')(cinputs)

        # 2 more inception layers
        #i1 = Convolution3D(48, 1, 1, 1, border_mode='same', subsample=(1, 1, 1), activation='relu',init='lecun_uniform')(cinputs)
        #i2_1 = Convolution3D(24, 1, 1, 1, border_mode='same', subsample=(1, 1, 1), activation='relu',init='lecun_uniform')(cinputs)
        #i2_2 = Convolution3D(96, 3, 3, 3, border_mode='same', subsample=(1, 1, 1), activation='relu',init='lecun_uniform')(i2_1)
        #i3_1 = Convolution3D(16, 1, 1, 1, border_mode='same', subsample=(1, 1, 1), activation='relu',init='lecun_uniform')(cinputs)
        #i3_2 = Convolution3D(64, 5, 5, 5, border_mode='same', subsample=(1, 1, 1), activation='relu',init='lecun_uniform')(i3_1)
        #i4_1 = MaxPooling3D(pool_size=(3, 3, 3), strides=(1, 1, 1), border_mode='same', dim_ordering='default')(cinputs)
        #i4_2 = Convolution3D(64, 1, 1, 1, border_mode='same', subsample=(1, 1, 1), activation='relu',init='lecun_uniform')(i4_1)
        #cinputs = merge([i1, i2_2, i3_2, i4_2], mode='concat', concat_axis=4)
        
        #i1 = Convolution3D(48, 1, 1, 1, border_mode='same', subsample=(1, 1, 1), activation='relu',init='lecun_uniform')(cinputs)
        #i2_1 = Convolution3D(24, 1, 1, 1, border_mode='same', subsample=(1, 1, 1), activation='relu',init='lecun_uniform')(cinputs)
        #i2_2 = Convolution3D(96, 3, 3, 3, border_mode='same', subsample=(1, 1, 1), activation='relu',init='lecun_uniform')(i2_1)
        #i3_1 = Convolution3D(16, 1, 1, 1, border_mode='same', subsample=(1, 1, 1), activation='relu',init='lecun_uniform')(cinputs)
        #i3_2 = Convolution3D(64, 5, 5, 5, border_mode='same', subsample=(1, 1, 1), activation='relu',init='lecun_uniform')(i3_1)
        #i4_1 = MaxPooling3D(pool_size=(3, 3, 3), strides=(1, 1, 1), border_mode='same', dim_ordering='default')(cinputs)
        #i4_2 = Convolution3D(64, 1, 1, 1, border_mode='same', subsample=(1, 1, 1), activation='relu',init='lecun_uniform')(i4_1)
        #cinputs = merge([i1, i2_2, i3_2, i4_2], mode='concat', concat_axis=4)
        
        # Average pooling
        #cinputs = AveragePooling3D(pool_size=(4, 4, 5), strides=(4, 4, 5), border_mode='valid', dim_ordering='default')(cinputs)
        
        f1 = Flatten()(cinputs)
        f1 = Dense(output_dim=16, activation='relu', init='lecun_uniform', W_regularizer=l2(0.0001))(f1)
        f1 = Dropout(.6)(f1)
        
        inc_output = Dense(output_dim=1, activation='sigmoid',init='normal', W_regularizer=l2(0.0005))(f1)
        incep = Model(inputs, inc_output)

        incep.compile(loss='binary_crossentropy',
                      optimizer=Nadam(lr=0.0005, beta_1=0.9, beta_2=0.999,
                                      epsilon=1e-08, schedule_decay=0.01), metrics=['accuracy'])
        
            #callbacks.EarlyStopping(monitor='val_loss', patience=20, mode='min'), 
    lcallbacks = [callbacks.ModelCheckpoint('models/conv3d_classifier_20.h5', monitor='val_loss', save_best_only=True, mode='min')]            
    incep.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 20, 20, 60, 1) 0                                            
____________________________________________________________________________________________________
convolution3d_1 (Convolution3D)  (None, 5, 5, 15, 256) 32256       input_1[0][0]                    
____________________________________________________________________________________________________
maxpooling3d_1 (MaxPooling3D)    (None, 3, 3, 8, 256)  0           convolution3d_1[0][0]            
____________________________________________________________________________________________________
batchnormalization_1 (BatchNorma (None, 3, 3, 8, 256)  512         maxpooling3d_1[0][0]             
___________________________________________________________________________________________

In [ ]:
hist = incep.fit(x_t, y_t, shuffle=True, nb_epoch=60, batch_size=100, verbose=1, validation_data=(x_v, y_v), callbacks=lcallbacks)

Train on 80000 samples, validate on 8000 samples
Epoch 1/60
80000/80000 [==============================] - 28s - loss: 0.8158 - acc: 0.5697 - val_loss: 0.6932 - val_acc: 0.5006
Epoch 2/60
80000/80000 [==============================] - 25s - loss: 0.7572 - acc: 0.6662 - val_loss: 0.6107 - val_acc: 0.6975
Epoch 3/60
80000/80000 [==============================] - 25s - loss: 0.7155 - acc: 0.6938 - val_loss: 0.5885 - val_acc: 0.7012
Epoch 4/60
80000/80000 [==============================] - 25s - loss: 0.6943 - acc: 0.7046 - val_loss: 0.5746 - val_acc: 0.7144
Epoch 5/60
80000/80000 [==============================] - 25s - loss: 0.6780 - acc: 0.7145 - val_loss: 0.5658 - val_acc: 0.7225
Epoch 6/60
80000/80000 [==============================] - 25s - loss: 0.6629 - acc: 0.7248 - val_loss: 0.5559 - val_acc: 0.7327
Epoch 7/60
80000/80000 [==============================] - 25s - loss: 0.6479 - acc: 0.7331 - val_loss: 0.5476 - val_acc: 0.7414
Epoch 8/60
80000/80000 [==============================]

In [8]:
## TEST EVENTS
f_start = 0
f_end = 25
# Read in the signal events.
print("Reading test signal events...")
for fn in range(f_start,f_end):
    print("-- Reading file {0}".format(fn))
    s_dat = tb.open_file("/home/jrenner/data/classification/bb_1M_v0_08_07/hdf5_maps_NEW_training_MC_si_{0}.h5".format(fn), 'r')
    if(fn == f_start):
        stest_array = np.array(s_dat.root.maps)
        stest_energies = np.array(s_dat.root.energies)
    else:
        stest_array = np.concatenate([stest_array,np.array(s_dat.root.maps)])
        stest_energies = np.concatenate([stest_energies,np.array(s_dat.root.energies)])
        
# Read in the background events.
print("Reading background events...")
for fn in range(f_start,f_end):
    print("-- Reading file {0}".format(fn))
    b_dat = tb.open_file("/home/jrenner/data/classification/se_1M_v0_08_07/hdf5_maps_NEW_training_MC_bg_{0}.h5".format(fn), 'r')
    if(fn == f_start):
        btest_array = np.array(b_dat.root.maps)
        btest_energies = np.array(b_dat.root.energies)
    else:
        btest_array = np.concatenate([btest_array,np.array(b_dat.root.maps)])
        btest_energies = np.concatenate([btest_energies,np.array(b_dat.root.energies)])

print("Read {0} test signal events and {1} test background events.".format(len(stest_array),len(btest_array)))

# Concatenate the datasets
print("Concatenating datasets...")
x_e = np.concatenate([stest_array, btest_array])
y_e = np.concatenate([np.ones([len(stest_array),1]), np.zeros([len(btest_array),1])])
x_e = np.reshape(x_e, (len(x_e), xdim, ydim, zdim, 1))

Reading test signal events...
-- Reading file 0
-- Reading file 1
-- Reading file 2
-- Reading file 3
-- Reading file 4
-- Reading file 5
-- Reading file 6
-- Reading file 7
-- Reading file 8
-- Reading file 9
-- Reading file 10
-- Reading file 11
-- Reading file 12
-- Reading file 13
-- Reading file 14
-- Reading file 15
-- Reading file 16
-- Reading file 17
-- Reading file 18
-- Reading file 19
-- Reading file 20
-- Reading file 21
-- Reading file 22
-- Reading file 23
-- Reading file 24
Reading background events...
-- Reading file 0
-- Reading file 1
-- Reading file 2
-- Reading file 3
-- Reading file 4
-- Reading file 5
-- Reading file 6
-- Reading file 7
-- Reading file 8
-- Reading file 9
-- Reading file 10
-- Reading file 11
-- Reading file 12
-- Reading file 13
-- Reading file 14
-- Reading file 15
-- Reading file 16
-- Reading file 17
-- Reading file 18
-- Reading file 19
-- Reading file 20
-- Reading file 21
-- Reading file 22
-- Reading file 23
-- Reading file 24
Read 47412 

In [10]:
loss_and_metrics = incep.evaluate(x_e, y_e);
y_pred = incep.predict(x_e, batch_size=100, verbose=0)
print(loss_and_metrics)

92064/92069 [============================>.] - ETA: 0s[0.44966127113524063, 0.80558059716082508]


In [36]:
for thh in [0.897818]:#np.arange(0,1,0.01):
    nts = 0; ntb = 0
    ncs = 0; ncb = 0
    for ye,yp in zip(y_e,y_pred):
        if(ye == 0):
            ntb += 1  # add one background event
            if(yp < thh):
                ncb += 1  # add one correctly predicted background event

        if(ye == 1):
            nts += 1  # add one signal event
            if(yp >= thh):
                ncs += 1  # add one correctly predicted signal event

    print("-- {0} of {1} ({2}%) correct background events; {3} of {4} ({5}%) correct signal events".format(ncb,ntb,1.0*ncb/ntb*100,ncs,nts,1.0*ncs/nts*100))

-- 43557 of 44657 (97.53678034798577%) correct background events; 11251 of 47412 (23.73027925419725%) correct signal events


In [ ]:
incep.save('models/largenet_noise.h5')

Notes:
- With of order 11k params, reached val loss of about 0.52 and seemed to slow significantly
- With of order 30k params, reached val loss of about 0.49 at best
- The problem of fluctuating validation loss was due to a learning rate that was too high

## Generation of 20x20 window table

In [ ]:
# Generate table consisting of 20x20 windows for each SiPM with maximum charge
# -- Table is 2304x400: for (i,j) in (0,0) to (48,48); the table contains the list 
#     of IDs for the corresponding 20x20 window for each 1D SiPM ID = i*48 + j

# Create the HDF5 file.
h5f = h5py.File("wtbl.h5")

tbl = np.zeros([48*48,400])
print("Generating window table...")
for i in range(48):
    for j in range(48):
        sipm_id = i*48 + j
        
        # Determine the 20x20 window.
        i_in = i - 10; i_fi = i + 10
        if(i_in < 0):
            i_fi = (i - i_in) + 10
            i_in = 0
        elif(i_fi > 48):
            i_in = i - (i_fi - 48) - 10
            i_fi = 48
        j_in = j - 10; j_fi = j + 10
        if(j_in < 0):
            j_fi = (j - j_in) + 10
            j_in = 0
        elif(j_fi > 48):
            j_in = j - (j_fi - 48) - 10
            j_fi = 48
            
        # Save the 20x20 window in the table.
        nwin = 0
        for iw in range(i_in,i_fi):
            for jw in range(j_in,j_fi):
                w_id = iw*48 + jw
                tbl[sipm_id][nwin] = w_id
                nwin += 1

# Save the table to an HDF5 file.
print("Saving table to file...")
h5f.create_dataset("wtbl",data=tbl)
h5f.close()

## Miscellaneous plots

In [ ]:
# Plot the energies of the training events.
esums = []
for earr in s_earray:
    esums.append(np.sum(earr)/12.)
#for earr in b_earray:
#    esums.append(np.sum(earr))
    
fig = plt.figure();
ax1 = fig.add_subplot(111);
fig.set_figheight(5.0)
fig.set_figwidth(7.5)

plt.hist(esums, 100, normed=1, facecolor='green')
plt.xlabel('Cathode charge')
plt.ylabel('Counts/bin')
plt.show()
print(b_earray[0])

In [ ]:
# Carried over from NEW_kr_diff_mc_train.ipynb
def NEW_SiPM_map_plot(xarr, yarr, plot_truth=True, normalize=True):
    """
    Plots a SiPM map in the NEW Geometry
    xarr is a NEW sipm map, yarr the pair of coordinates the map corresponds to
    """
    if normalize:
        probs = (xarr - np.min(xarr))
        probs /= np.max(probs)
    else: 
        probs = xarr

    fig = plt.figure();
    ax1 = fig.add_subplot(111);
    fig.set_figheight(7.0)
    fig.set_figwidth(7.0)
    ax1.axis([-250, 250, -250, 250]);

    for i in range(48):
        for j in range(48):
            r = Ellipse(xy=(i * 10 - 235, j * 10 - 235), width=2., height=2.);
            r.set_facecolor('0');
            r.set_alpha(probs[i, j]);
            ax1.add_artist(r);
            
    if plot_truth:
        # Place a large blue circle for actual EL points.
        xpt = yarr[0]
        ypt = yarr[1]
        mrk = Ellipse(xy=(xpt,ypt), width=4., height=4.);
        mrk.set_facecolor('b');
        ax1.add_artist(mrk);
        #print(xpt,ypt)
        
    plt.xlabel("x (mm)");
    plt.ylabel("y (mm)");

In [ ]:
# Plot training event slices.
plt_nevt = 14900
plt_nslice = 5

plt_arr = x_t[plt_nevt,:,:,plt_nslice]
NEW_SiPM_map_plot(plt_arr,[0, 0], False)
chg_sum = np.sum(plt_arr)
tot_chg_sum = np.sum(x_t[plt_nevt,:,:,:])
max_chg = np.max(x_t[plt_nevt,:,:,plt_nslice])
min_chg = np.min(x_t[plt_nevt,:,:,plt_nslice])
print("Plotting event", plt_nevt, "slice", plt_nslice, "with charge sum", chg_sum, "and total sum", tot_chg_sum,
     "max charge", max_chg, "and min charge", min_chg)

In [ ]:
# Return a histogram containing the SiPM distribution for a given (x,y,z), where x,y,z are indices in the map.
def SiPM_dist(sipm_maps,x,y,z):
    
    return sipm_maps[:,x,y,z]

In [ ]:
# Determine which SiPMs have all-zero distributions for a given slice.
slnum = 0
for sipmx in range(xdim):
    for sipmy in range(ydim):
        
        dist = x_t[:,sipmx,sipmy,slnum]
        nzeros = len(np.nonzero(dist)[0])
        if(nzeros == 0):
            print("All zeros for SiPM ({0},{1})".format(sipmx,sipmy))
    

In [ ]:
# Plot 12 distributions.
fig = plt.figure();
fig.set_figheight(20.0)
fig.set_figwidth(13.0)

for ndist in range(3):
    sp = int(430 + ndist + 1)
    print("plot",sp)
    ax = fig.add_subplot(sp);
    
    xv = np.random.randint(38) + 5
    yv = np.random.randint(38) + 5 
    sv = np.random.randint(14)

    dist = x_t[:,xv,yv,sv]
    plt.hist(dist[np.nonzero(dist)], 1000, normed=0, facecolor='green')
    ax.set_yscale('log')
    start, end = ax.get_xlim()
    #ax.xaxis.set_ticks(np.arange(start, end, (end-start)/4.))
    plt.xlim(0.0,0.05)
    plt.xlabel('SiPM Charge')
    plt.ylabel('Counts/bin')
    plt.title('SiPM ({0},{1}); slice {2}'.format(xv,yv,sv))
    
plt.show()

In [ ]:
# Plot SiPM sum distributions for a range of events.
#tr_evt_no = 15
fig = plt.figure();
fig.set_figheight(5.0)
fig.set_figwidth(5.0)

s_values = []
for tr_evt_no in range(1000):
    for nx in range(xdim):
        for ny in range(ydim):
            sval = np.sum(x_t[tr_evt_no,nx,ny,:])
            if(sval != 0):
                s_values.append(sval)
        #print("SiPM at ({0},{1})".format(nx,ny))

ax = fig.add_subplot(111);
plt.hist(s_values, 100, normed=0, facecolor='green')
ax.set_yscale('log')
#start, end = ax.get_xlim()
#ax.xaxis.set_ticks(np.arange(start, end, (end-start)/4.))
plt.xlim(0.0,0.3)
plt.xlabel('SiPM Charge Sum')
plt.ylabel('Counts/bin')
#plt.title('Event {0}'.format(tr_evt_no))

In [ ]:
print(len(x_t))

In [ ]:
# Old nets
        cinputs = Convolution3D(32, 6, 6, 6, border_mode='same', subsample=(3, 3, 5), activation='relu',init='normal', W_regularizer=l2(0.001))(inputs)
        cinputs = MaxPooling3D(pool_size=(3, 3, 3), strides=(2, 2, 3), border_mode='same', dim_ordering='default')(cinputs)
        cinputs = BatchNormalization(epsilon=1e-05, mode=0, axis=4, momentum=0.99, weights=None, beta_init='zero', gamma_init='one', gamma_regularizer=None, beta_regularizer=None)(cinputs)
        cinputs = Convolution3D(16, 1, 1, 1, border_mode='same', subsample=(1, 1, 1), activation='relu',init='normal', W_regularizer=l2(0.001))(cinputs)
        cinputs = Convolution3D(32, 2, 2, 2, border_mode='same', subsample=(1, 1, 1), activation='relu',init='normal', W_regularizer=l2(0.001))(cinputs)
        cinputs = BatchNormalization(epsilon=1e-05, mode=0, axis=4, momentum=0.99, weights=None, beta_init='zero', gamma_init='one', gamma_regularizer=None, beta_regularizer=None)(cinputs)
        cinputs = MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), border_mode='same', dim_ordering='default')(cinputs)
        
        

        inputs = Input(shape=(48, 48, 30, 1))
        cinputs = Convolution3D(512, 6, 6, 6, border_mode='same', subsample=(3, 3, 5), activation='relu',init='normal', W_regularizer=l2(0.001))(inputs)
        cinputs = MaxPooling3D(pool_size=(3, 3, 3), strides=(2, 2, 3), border_mode='same', dim_ordering='default')(cinputs)
        cinputs = BatchNormalization(epsilon=1e-05, mode=0, axis=4, momentum=0.99, weights=None, beta_init='zero', gamma_init='one', gamma_regularizer=None, beta_regularizer=None)(cinputs)
        cinputs = Convolution3D(64, 1, 1, 1, border_mode='same', subsample=(1, 1, 1), activation='relu',init='normal', W_regularizer=l2(0.001))(cinputs)
        cinputs = Convolution3D(1024, 2, 2, 2, border_mode='same', subsample=(1, 1, 1), activation='relu',init='normal', W_regularizer=l2(0.001))(cinputs)
        cinputs = BatchNormalization(epsilon=1e-05, mode=0, axis=4, momentum=0.99, weights=None, beta_init='zero', gamma_init='one', gamma_regularizer=None, beta_regularizer=None)(cinputs)
        cinputs = MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), border_mode='same', dim_ordering='default')(cinputs)
        cinputs = Convolution3D(16, 1, 1, 1, border_mode='same', subsample=(1, 1, 1), activation='relu',init='normal', W_regularizer=l2(0.001))(cinputs)
        #cinputs = Convolution3D(16, 3, 3, 3, border_mode='same', subsample=(2, 2, 2), activation='relu',init='normal')(cinputs)
        #cinputs = Convolution3D(32, 3, 3, 3, border_mode='same', subsample=(2, 2, 5), activation='relu',init='normal')(cinputs)
        #cinputs = Convolution3D(64, 2, 2, 2, border_mode='same', subsample=(2, 2, 1), activation='relu',init='normal')(cinputs)
        #cinputs = Convolution3D(512, 2, 2, 2, border_mode='same', subsample=(2, 2, 5), activation='relu',init='normal')(cinputs)
        #cinputs = Convolution3D(1024, 2, 2, 1, border_mode='same', subsample=(1, 1, 1), activation='relu',init='normal')(cinputs)
        #cinputs = Convolution3D(1024, 2, 2, 1, border_mode='valid', subsample=(3, 3, 1), activation='relu')(cinputs)
        f1 = Flatten()(cinputs)
        f1 = Dense(output_dim=1024, activation='relu', init='normal', W_regularizer=l2(0.001))(f1)
        f1 = Dropout(.4)(f1)

        inc_output = Dense(output_dim=1, activation='sigmoid',init='normal')(f1)
        incep = Model(inputs, inc_output)
        
        
        

        cinputs = Convolution3D(32, 6, 6, 6, border_mode='same', subsample=(3, 3, 5), activation='sigmoid',init='normal', W_regularizer=l2(0.001))(inputs)
        cinputs = MaxPooling3D(pool_size=(3, 3, 3), strides=(2, 2, 3), border_mode='same', dim_ordering='default')(cinputs)
        #cinputs = BatchNormalization(epsilon=1e-05, mode=0, axis=4, momentum=0.99, weights=None, beta_init='zero', gamma_init='one', gamma_regularizer=None, beta_regularizer=None)(cinputs)
        #cinputs = Convolution3D(64, 1, 1, 1, border_mode='same', subsample=(1, 1, 1), activation='tanh',init='normal')(cinputs)
        cinputs = Convolution3D(64, 2, 2, 2, border_mode='same', subsample=(1, 1, 1), activation='sigmoid',init='normal', W_regularizer=l2(0.001))(cinputs)
        #cinputs = BatchNormalization(epsilon=1e-05, mode=0, axis=4, momentum=0.99, weights=None, beta_init='zero', gamma_init='one', gamma_regularizer=None, beta_regularizer=None)(cinputs)
        cinputs = MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), border_mode='same', dim_ordering='default')(cinputs)
        #cinputs = Convolution3D(16, 1, 1, 1, border_mode='same', subsample=(1, 1, 1), activation='tanh',init='normal')(cinputs)
        #cinputs = Convolution3D(16, 3, 3, 3, border_mode='same', subsample=(2, 2, 2), activation='relu',init='normal')(cinputs)
        #cinputs = Convolution3D(32, 3, 3, 3, border_mode='same', subsample=(2, 2, 5), activation='relu',init='normal')(cinputs)
        #cinputs = Convolution3D(64, 2, 2, 2, border_mode='same', subsample=(2, 2, 1), activation='relu',init='normal')(cinputs)
        #cinputs = Convolution3D(512, 2, 2, 2, border_mode='same', subsample=(2, 2, 5), activation='relu',init='normal')(cinputs)
        #cinputs = Convolution3D(1024, 2, 2, 1, border_mode='same', subsample=(1, 1, 1), activation='relu',init='normal')(cinputs)
        #cinputs = Convolution3D(1024, 2, 2, 1, border_mode='valid', subsample=(3, 3, 1), activation='relu')(cinputs)

        inputs = Input(shape=(48, 48, 30))
        cinputs = Convolution2D(512, 6, 6, border_mode='same', subsample=(3, 3), activation='relu',init='normal', W_regularizer=l2(0.001))(inputs)
        cinputs = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), border_mode='same', dim_ordering='default')(cinputs)
        cinputs = BatchNormalization(epsilon=1e-05, mode=0, axis=3, momentum=0.99, weights=None, beta_init='zero', gamma_init='one', gamma_regularizer=None, beta_regularizer=None)(cinputs)
        cinputs = Convolution2D(64, 1, 1, border_mode='same', subsample=(1, 1), activation='relu',init='normal', W_regularizer=l2(0.001))(cinputs)
        cinputs = Convolution2D(1024, 2, 2, border_mode='same', subsample=(1, 1), activation='relu',init='normal', W_regularizer=l2(0.001))(cinputs)
        cinputs = BatchNormalization(epsilon=1e-05, mode=0, axis=4, momentum=0.99, weights=None, beta_init='zero', gamma_init='one', gamma_regularizer=None, beta_regularizer=None)(cinputs)
        cinputs = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), border_mode='same', dim_ordering='default')(cinputs)
        cinputs = Convolution2D(16, 1, 1, border_mode='same', subsample=(1, 1), activation='relu',init='normal', W_regularizer=l2(0.001))(cinputs)
        
        # 2D net 16-12-16
        inputs = Input(shape=(48, 48, 30))
        cinputs = Convolution2D(32, 6, 6, border_mode='same', subsample=(3, 3), activation='relu',init='lecun_uniform', W_regularizer=l2(0.01))(inputs)
        cinputs = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), border_mode='same', dim_ordering='default')(cinputs)
        cinputs = BatchNormalization(epsilon=1e-05, mode=0, axis=3, momentum=0.99, weights=None, beta_init='zero', gamma_init='one', gamma_regularizer=None, beta_regularizer=None)(cinputs)
        #cinputs = Convolution2D(8, 1, 1, border_mode='same', subsample=(1, 1), activation='relu',init='lecun_uniform', W_regularizer=l2(0.05))(cinputs)
        cinputs = Convolution2D(64, 2, 2, border_mode='same', subsample=(1, 1), activation='relu',init='lecun_uniform', W_regularizer=l2(0.01))(cinputs)
        cinputs = BatchNormalization(epsilon=1e-05, mode=0, axis=3, momentum=0.99, weights=None, beta_init='zero', gamma_init='one', gamma_regularizer=None, beta_regularizer=None)(cinputs)
        cinputs = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), border_mode='same', dim_ordering='default')(cinputs)
        cinputs = Convolution2D(128, 1, 1, border_mode='same', subsample=(1, 1), activation='relu',init='lecun_uniform', W_regularizer=l2(0.01))(cinputs)
        cinputs = BatchNormalization(epsilon=1e-05, mode=0, axis=3, momentum=0.99, weights=None, beta_init='zero', gamma_init='one', gamma_regularizer=None, beta_regularizer=None)(cinputs)
        f1 = Flatten()(cinputs)
        f1 = Dense(output_dim=128, activation='relu', init='lecun_uniform', W_regularizer=l2(0.008))(f1)
        f1 = Dropout(.6)(f1)

        inc_output = Dense(output_dim=1, activation='sigmoid',init='lecun_uniform', W_regularizer=l2(0.001))(f1)
        incep = Model(inputs, inc_output)
        
        # Good 3D net 16-12-16
        inputs = Input(shape=(48, 48, 30, 1))
        cinputs = Convolution3D(128, 6, 6, 6, border_mode='same', subsample=(3, 3, 5), activation='relu',init='lecun_uniform', W_regularizer=l2(0.01))(inputs)
        cinputs = MaxPooling3D(pool_size=(3, 3, 3), strides=(2, 2, 3), border_mode='same', dim_ordering='default')(cinputs)
        cinputs = BatchNormalization(epsilon=1e-05, mode=0, axis=4, momentum=0.99, weights=None, beta_init='zero', gamma_init='one', gamma_regularizer=None, beta_regularizer=None)(cinputs)
        cinputs = Convolution3D(64, 1, 1, 1, border_mode='same', subsample=(1, 1, 1), activation='relu',init='lecun_uniform', W_regularizer=l2(0.01))(cinputs)
        cinputs = Convolution3D(128, 2, 2, 2, border_mode='same', subsample=(1, 1, 1), activation='relu',init='lecun_uniform', W_regularizer=l2(0.01))(cinputs)
        cinputs = BatchNormalization(epsilon=1e-05, mode=0, axis=4, momentum=0.99, weights=None, beta_init='zero', gamma_init='one', gamma_regularizer=None, beta_regularizer=None)(cinputs)
        cinputs = MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), border_mode='same', dim_ordering='default')(cinputs)
        #cinputs = Convolution3D(16, 1, 1, 1, border_mode='same', subsample=(1, 1, 1), activation='relu',init='lecun_uniform', W_regularizer=l2(0.01))(cinputs)
        #cinputs = Convolution3D(16, 3, 3, 3, border_mode='same', subsample=(2, 2, 2), activation='relu',init='normal')(cinputs)
        #cinputs = Convolution3D(32, 3, 3, 3, border_mode='same', subsample=(2, 2, 5), activation='relu',init='normal')(cinputs)
        #cinputs = Convolution3D(64, 2, 2, 2, border_mode='same', subsample=(2, 2, 1), activation='relu',init='normal')(cinputs)
        #cinputs = Convolution3D(512, 2, 2, 2, border_mode='same', subsample=(2, 2, 5), activation='relu',init='normal')(cinputs)
        #cinputs = Convolution3D(1024, 2, 2, 1, border_mode='same', subsample=(1, 1, 1), activation='relu',init='normal')(cinputs)
        #cinputs = Convolution3D(1024, 2, 2, 1, border_mode='valid', subsample=(3, 3, 1), activation='relu')(cinputs)
        f1 = Flatten()(cinputs)
        f1 = Dense(output_dim=64, activation='relu', init='lecun_uniform', W_regularizer=l2(0.02))(f1)
        f1 = Dropout(.7)(f1)
        
        
        
        # DNN for 20x20x30
        inputs = Input(shape=(xdim, ydim, zdim, 1))
        cinputs = Convolution3D(256, 5, 5, 5, border_mode='same', subsample=(4, 4, 3), activation='relu',init='lecun_uniform', W_regularizer=l2(0.001))(inputs)
        cinputs = MaxPooling3D(pool_size=(3, 3, 3), strides=(2, 2, 2), border_mode='same', dim_ordering='default')(cinputs)
        cinputs = BatchNormalization(epsilon=1e-05, mode=0, axis=4, momentum=0.99, weights=None, beta_init='zero', gamma_init='one', gamma_regularizer=None, beta_regularizer=None)(cinputs)
        cinputs = Convolution3D(64, 1, 1, 1, border_mode='same', subsample=(1, 1, 1), activation='relu',init='lecun_uniform', W_regularizer=l2(0.001))(cinputs)
        cinputs = Convolution3D(128, 2, 2, 3, border_mode='same', subsample=(2, 2, 3), activation='relu',init='lecun_uniform', W_regularizer=l2(0.001))(cinputs)
        cinputs = BatchNormalization(epsilon=1e-05, mode=0, axis=4, momentum=0.99, weights=None, beta_init='zero', gamma_init='one', gamma_regularizer=None, beta_regularizer=None)(cinputs)
        cinputs = Convolution3D(128, 1, 1, 1, border_mode='same', subsample=(1, 1, 1), activation='relu',init='lecun_uniform', W_regularizer=l2(0.001))(cinputs)
        cinputs = MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), border_mode='same', dim_ordering='default')(cinputs)
        f1 = Flatten()(cinputs)
        f1 = Dense(output_dim=128, activation='relu', init='lecun_uniform', W_regularizer=l2(0.001))(f1)
        f1 = Dropout(.7)(f1)

        inc_output = Dense(output_dim=1, activation='sigmoid',init='normal', W_regularizer=l2(0.001))(f1)
        incep = Model(inputs, inc_output)

        incep.compile(loss='binary_crossentropy',
                      optimizer=Nadam(lr=0.00001, beta_1=0.9, beta_2=0.999,
                                      epsilon=1e-08, schedule_decay=0.1), metrics=['accuracy'])

In [ ]:
# Add noise
nsigma = 1.0e-4

inoise = x_t == 0.
x_t[inoise] = np.abs(np.random.normal(0,1.0e-4,np.sum(inoise)))

inoise = x_v == 0.
x_v[inoise] = np.abs(np.random.normal(0,1.0e-4,np.sum(inoise)))

In [ ]:
# OLD READ METHOD

# Signal events.
s_dat = tb.open_file('/home/jrenner/data/classification/NEW_training_MC_si_20.h5', 'r')
#s_dat = tb.open_file('/home/jrenner/data/classification/NEW_training_MC_si_nst_nonorm.h5', 'r')
print(s_dat)
s_array = np.array(s_dat.root.maps)
x_t = s_array[:Ntrain]
x_v = s_array[Ntrain:Ntot-Ntest]
x_e = s_array[Ntot-Ntest:Ntot]
y_t = np.ones([Ntrain, 1])
y_v = np.ones([Ntot-Ntrain-Ntest, 1])
y_e = np.ones([Ntest, 1])

s_earray = np.array(s_dat.root.energies)

# Background events.
b_dat = tb.open_file('/home/jrenner/data/classification/NEW_training_MC_bg_20.h5', 'r')
#b_dat = tb.open_file('/home/jrenner/data/classification/NEW_training_MC_bg_nst_nonorm.h5', 'r')
print(b_dat)
b_array = np.array(b_dat.root.maps)
print("Concatenating datasets...")
x_t = np.concatenate([x_t, b_array[:Ntrain]])
x_v = np.concatenate([x_v, b_array[Ntrain:Ntot-Ntest]])
x_e = np.concatenate([x_e, b_array[Ntot-Ntest:Ntot]])
y_bt = np.zeros([Ntrain, 1])
y_t = np.concatenate([y_t, y_bt])
y_bv = np.zeros([Ntot-Ntrain-Ntest, 1])
y_v = np.concatenate([y_v, y_bv])
y_be = np.zeros([Ntest, 1])
y_e = np.concatenate([y_e, y_be])

b_earray = np.array(b_dat.root.energies)

# Normalize
#mval = max(np.max(s_array),np.max(b_array))
#muval = np.mean(s_array)
#sval = np.std(s_array)
#print("Normalizing with max value of", mval, "(mean of", muval, "; sigma of ", sval, ")")
#x_t /= sval
#x_v /= sval